In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 1. Dataset (Example 1 wala hi use kar rahe hain)
data = {
    'StudyHours': ['Low', 'High', 'High', 'Low', 'High', 'Low', 'High', 'Low'],
    'Attendance': ['Poor', 'Good', 'Poor', 'Good', 'Good', 'Poor', 'Good', 'Poor'],
    'Result': ['Fail', 'Pass', 'Pass', 'Fail', 'Pass', 'Fail', 'Pass', 'Fail']
}
df = pd.DataFrame(data)
encoder = LabelEncoder()
for col in df.columns:
    df[col] = encoder.fit_transform(df[col])

X = df[['StudyHours', 'Attendance']].values
y = df['Result'].values

# --- Shallow ANN (1 Hidden Layer) ---
model_shallow = Sequential([
    Dense(8, activation='relu', input_shape=(2,)),
    Dense(1, activation='sigmoid')
])
model_shallow.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_shallow.fit(X, y, epochs=100, verbose=0)
_, acc_shallow = model_shallow.evaluate(X, y, verbose=0)

# --- Deep DNN (3 Hidden Layers) ---
model_deep = Sequential([
    Dense(16, activation='relu', input_shape=(2,)),
    Dense(12, activation='relu'),
    Dense(8, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_deep.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_deep.fit(X, y, epochs=100, verbose=0)
_, acc_deep = model_deep.evaluate(X, y, verbose=0)

print(f"Shallow ANN Accuracy: {acc_shallow*100:.2f}%")
print(f"Deep DNN Accuracy: {acc_deep*100:.2f}%")

e:\University\Semster 5\AI Lab\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Shallow ANN Accuracy: 12.50%
Deep DNN Accuracy: 87.50%


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, to_categorical(iris.target), test_size=0.2)

def build_model(activation_func):
    model = Sequential([
        Dense(16, activation=activation_func, input_shape=(4,)),
        Dense(12, activation=activation_func),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model A (ReLU)
model_relu = build_model('relu')
history_relu = model_relu.fit(X_train, y_train, epochs=100, verbose=0)

# Model B (Tanh)
model_tanh = build_model('tanh')
history_tanh = model_tanh.fit(X_train, y_train, epochs=100, verbose=0)

print(f"ReLU Accuracy: {model_relu.evaluate(X_test, y_test, verbose=0)[1]*100:.2f}%")
print(f"Tanh Accuracy: {model_tanh.evaluate(X_test, y_test, verbose=0)[1]*100:.2f}%")

# Observation: ReLU aksar fast seekhta hai, jabke Tanh zero-centered hota hai aur smoothly converge karta hai.

ReLU Accuracy: 93.33%
Tanh Accuracy: 96.67%


In [3]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784) / 255.0
X_test = X_test.reshape(-1, 784) / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Model with more neurons and layers
model_tuned = Sequential([
    Dense(512, activation='relu', input_shape=(784,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model_tuned.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Experimenting with Batch Size
import time
start = time.time()
model_tuned.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1) # Smaller batch = slower but more updates
end = time.time()

print(f"Training Time with Batch Size 32: {end - start:.2f} seconds")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 10s 1us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.9396 - loss: 0.1964
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9737 - loss: 0.0868
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9811 - loss: 0.0623
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.9852 - loss: 0.0475
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9876 - loss: 0.0390
Training Time with Batch Size 32: 85.41 seconds


In [4]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Model without Regularization (Slightly Overfit)
model_overfit = Sequential([
    Dense(512, activation='relu', input_shape=(784,)),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])
model_overfit.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model WITH Dropout and Early Stopping
model_reg = Sequential([
    Dense(512, activation='relu', input_shape=(784,)),
    Dropout(0.3), # Dropout added
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

# Early stopping set karna
early_stop = EarlyStopping(monitor='val_loss', patience=3)

model_reg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_reg = model_reg.fit(X_train, y_train, epochs=20, validation_split=0.2, callbacks=[early_stop])

print("Regularization applied successfully to prevent overfitting.")

Epoch 1/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.9202 - loss: 0.2589 - val_accuracy: 0.9612 - val_loss: 0.1318
Epoch 2/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.9609 - loss: 0.1287 - val_accuracy: 0.9714 - val_loss: 0.0952
Epoch 3/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.9686 - loss: 0.0997 - val_accuracy: 0.9737 - val_loss: 0.0898
Epoch 4/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.9722 - loss: 0.0903 - val_accuracy: 0.9748 - val_loss: 0.0871
Epoch 5/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.9774 - loss: 0.0737 - val_accuracy: 0.9784 - val_loss: 0.0837
Epoch 6/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.9789 - loss: 0.0691 - val_accuracy: 0.9769 - val_loss: 0.0835
Epoch 7/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.9817 - loss: 0.0593 - val_accuracy: 0.9777 - val_loss: 0.0914
Epoch 8/20
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.9808 -